In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

In [27]:
df=pd.read_csv("C:/Users/dell/Downloads/customer_churn.csv")
df.shape

(7043, 21)

In [68]:
df.loc[4]

gender                        Female
SeniorCitizen                      0
Partner                           No
Dependents                        No
tenure                             2
PhoneService                     Yes
MultipleLines                     No
InternetService          Fiber optic
OnlineSecurity                    No
OnlineBackup                      No
DeviceProtection                  No
TechSupport                       No
StreamingTV                       No
StreamingMovies                   No
Contract              Month-to-month
PaperlessBilling                 Yes
PaymentMethod       Electronic check
MonthlyCharges                  70.7
TotalCharges                  151.65
Churn                            Yes
Name: 4, dtype: object

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


insights
1. there is no nan values
2. all columns data type is ok but stizens data type is rong
3. many columns are int formate of yes and no
4. customer id column is us less 
5. total charges in object data ttype with missing values ' '

In [30]:
df.drop(columns='customerID',inplace=True)
df['TotalCharges'] = df['TotalCharges'].str.replace(' ','0').astype(float)
df.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
3874,Female,0,No,No,9,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,43.75,405.70,No
4826,Female,1,No,No,8,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,105.50,829.55,Yes
985,Female,1,Yes,No,33,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),80.60,2656.50,Yes
5120,Male,0,Yes,Yes,7,Yes,No,DSL,Yes,No,No,No,No,No,One year,Yes,Electronic check,49.75,331.30,Yes
1792,Male,0,Yes,Yes,42,Yes,Yes,DSL,Yes,Yes,No,No,Yes,No,Month-to-month,Yes,Bank transfer (automatic),69.40,3058.30,No


In [31]:
df.to_csv('filterd_churn_data.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [32]:

import pandas as pd
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Churn Dataset Report", explorative=True)
profile.to_file("churn_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:01<00:00, 19.70it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:

cat_columns= df.select_dtypes(include=['object'])
print(cat_columns.shape)
num_columns= df.drop(columns=cat_columns)
num_columns_p=num_columns.drop(columns=['SeniorCitizen']).columns
cat_columns_p=cat_columns.drop(columns=['Contract']).columns
print(num_columns_p)
print(cat_columns_p)



(7043, 16)
Index(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')
Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
       'PaymentMethod', 'Churn'],
      dtype='object')


analisis to num columns

In [34]:

# def linear_num(column_data,column_name):
#     plt.scatter(column_data, df['Churn'], alpha=0.4)
#     plt.xlabel(f'{column_name}')
#     plt.ylabel('Churn')
#     plt.title(f'{column_name} vs Churn')
#     plt.show()




def hist_num(column_data,column_name):
    sns.histplot(column_data,kde=True,bins=20)
    plt.grid(True, linestyle='--',linewidth=0.5)
    plt.xlabel(f'distribution by {column_name}')
    plt.ylabel('count')
    plt.show()
    
    print('*'*90)
    print(column_data.describe())
    print('*'*90)
for col in num_columns:
    hist_num(df[col],col)
    

C:\Users\dell\AppData\Local\Temp\ipykernel_19388\522149934.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count    7043.000000
mean        0.162147
std         0.368612
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: SeniorCitizen, dtype: float64
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\522149934.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\522149934.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count    7043.000000
mean       64.761692
std        30.090047
min        18.250000
25%        35.500000
50%        70.350000
75%        89.850000
max       118.750000
Name: MonthlyCharges, dtype: float64
******************************************************************************************
******************************************************************************************
count    7043.000000
mean     2279.734304
std      2266.794470
min         0.000000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\522149934.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [35]:
def hist_num(column_data,column_name):
    sns.boxplot(column_data)
    plt.grid(True, linestyle='--',linewidth=0.5)
    plt.xlabel(f'distribution by {column_name}')
    plt.ylabel('count')
    plt.show()
    
    print('*'*90)
    print(column_data.describe())
    print('*'*90)
for col in num_columns:
    hist_num(df[col],col)

C:\Users\dell\AppData\Local\Temp\ipykernel_19388\2609507741.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\2609507741.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count    7043.000000
mean        0.162147
std         0.368612
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: SeniorCitizen, dtype: float64
******************************************************************************************
******************************************************************************************
count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64
******************************************************************************************
******************************************************************************************
count    7043.000000
mean       64.761692
std        30.090047
min        18.250000
25%        35.500000
50%        70.350000
75%        89.8500

C:\Users\dell\AppData\Local\Temp\ipykernel_19388\2609507741.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count    7043.000000
mean     2279.734304
std      2266.794470
min         0.000000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\2609507741.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


insights
1. numerical data is skewed
2. no outliers

analisis of catagorical columns

In [36]:
def hist_num(column_data,column_name):
    plt.figure(figsize=(5,3))
    sns.countplot(x=column_data)
    plt.grid(True, linestyle='--',linewidth=0.5)
    plt.xlabel(f'distribution by {column_name}')
    plt.ylabel('count')
    plt.show()
    
    print('*'*90)
    print(column_data.describe())
    print('*'*90)
for col in cat_columns:
    hist_num(df[col],col)

C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count     7043
unique       2
top       Male
freq      3555
Name: gender, dtype: object
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count     7043
unique       2
top         No
freq      3641
Name: Partner, dtype: object
******************************************************************************************
******************************************************************************************
count     7043
unique       2
top         No
freq      4933
Name: Dependents, dtype: object
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


...

count     7043
unique       3
top         No
freq      3473
Name: TechSupport, dtype: object
******************************************************************************************
******************************************************************************************
count     7043
unique       3
top         No
freq      2810
Name: StreamingTV, dtype: object
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(5,3))


******************************************************************************************
count     7043
unique       3
top         No
freq      2785
Name: StreamingMovies, dtype: object
******************************************************************************************
******************************************************************************************
count               7043
unique                 3
top       Month-to-month
freq                3875
Name: Contract, dtype: object
******************************************************************************************


C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\dell\AppData\Local\Temp\ipykernel_19388\4268680270.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


******************************************************************************************
count     7043
unique       2
top        Yes
freq      4171
Name: PaperlessBilling, dtype: object
******************************************************************************************
******************************************************************************************
count                 7043
unique                   4
top       Electronic check
freq                  2365
Name: PaymentMethod, dtype: object
******************************************************************************************
******************************************************************************************
count     7043
unique       2
top         No
freq      5174
Name: Churn, dtype: object
******************************************************************************************


1. maximum customers are happy with paperless billing
2. month to moth is most frequent plan


In [37]:
for col in cat_columns:
    print(f'{col} =>  {df[col].unique()}')
    print('#'*30)

gender =>  ['Female' 'Male']
##############################
Partner =>  ['Yes' 'No']
##############################
Dependents =>  ['No' 'Yes']
##############################
PhoneService =>  ['No' 'Yes']
##############################
MultipleLines =>  ['No phone service' 'No' 'Yes']
##############################
InternetService =>  ['DSL' 'Fiber optic' 'No']
##############################
OnlineSecurity =>  ['No' 'Yes' 'No internet service']
##############################
OnlineBackup =>  ['Yes' 'No' 'No internet service']
##############################
DeviceProtection =>  ['No' 'Yes' 'No internet service']
##############################
TechSupport =>  ['No' 'Yes' 'No internet service']
##############################
StreamingTV =>  ['No' 'Yes' 'No internet service']
##############################
StreamingMovies =>  ['No' 'Yes' 'No internet service']
##############################
Contract =>  ['Month-to-month' 'One year' 'Two year']
##############################
PaperlessBillin

In [38]:
yn_col=[]
for col in cat_columns:
    if df[col].nunique()==2:
        yn_col.append(col)

mult_col=cat_columns.drop(columns=yn_col)
yn_col=df[yn_col]
print(yn_col.nunique())
print('*'*20)
print(mult_col.nunique())


gender              2
Partner             2
Dependents          2
PhoneService        2
PaperlessBilling    2
Churn               2
dtype: int64
********************
MultipleLines       3
InternetService     3
OnlineSecurity      3
OnlineBackup        3
DeviceProtection    3
TechSupport         3
StreamingTV         3
StreamingMovies     3
Contract            3
PaymentMethod       4
dtype: int64


In [39]:
df['Contract']

0       Month-to-month
1             One year
2       Month-to-month
3             One year
4       Month-to-month
             ...      
7038          One year
7039          One year
7040    Month-to-month
7041    Month-to-month
7042          Two year
Name: Contract, Length: 7043, dtype: object

In [40]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [41]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [42]:
x=df.drop(columns=['Churn'])
y=df['Churn'].apply( lambda x : 1 if x=='Yes' else 0)
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [43]:

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)
print(x_train.columns.unique)

<bound method Index.unique of Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')>


In [44]:


for col in x_train.columns:
    print(f'{col} =>  {df[col].unique()}')
    print('#'*30)

gender =>  ['Female' 'Male']
##############################
SeniorCitizen =>  [0 1]
##############################
Partner =>  ['Yes' 'No']
##############################
Dependents =>  ['No' 'Yes']
##############################
tenure =>  [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
##############################
PhoneService =>  ['No' 'Yes']
##############################
MultipleLines =>  ['No phone service' 'No' 'Yes']
##############################
InternetService =>  ['DSL' 'Fiber optic' 'No']
##############################
OnlineSecurity =>  ['No' 'Yes' 'No internet service']
##############################
OnlineBackup =>  ['Yes' 'No' 'No internet service']
##############################
DeviceProtection =>  ['No' 'Yes' 'No internet service']
##############################
TechSupport =>  ['No' 'Yes' '

In [63]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.ensemble import VotingClassifier
from collections import Counter
from xgboost import XGBClassifier



or_catagory=[['Month-to-month','One year' ,'Two year']]
num_pipe= Pipeline(steps=[
    ('scale',StandardScaler()
    )])
list_numcol=['tenure',  'MonthlyCharges','MonthlyCharges',]
list_catcol=['gender','Partner', 'Dependents', 'PhoneService','MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'PaperlessBilling', 'PaymentMethod']
lable_pipe=Pipeline(
    steps=[('label',OrdinalEncoder(categories=or_catagory))
    ])
cat_pipe=Pipeline(
    steps=[
        ('encode',OneHotEncoder(drop='first',sparse_output=False)),])


process=ColumnTransformer(
    transformers=[
        ('num',num_pipe,list_numcol),
        ('lab', lable_pipe,['Contract']),
        ('cat',cat_pipe,list_catcol)
        
    ])



x_train_processed = process.fit_transform(x_train)

smote = SMOTE(random_state=42)
x_train_res, y_train_res = smote.fit_resample(x_train_processed, y_train)




counter = Counter(y_train_res)
ratio = counter[0] / counter[1]  
model_xgboost = XGBClassifier(scale_pos_weight=ratio)#
model_rf=  RandomForestClassifier(class_weight="balanced")
model_svm=SVC(class_weight="balanced")


from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[
    ('rf', model_rf), ('xg', model_xgboost), ('svc', model_svm)],
    voting='hard')  


voting_clf.fit(x_train_res, y_train_res)


process= Pipeline(steps=[
    ("process",process)])

model = Pipeline(steps=[
    
    ("classifier",voting_clf)
])
scores = cross_val_score(voting_clf ,x_train_res, y_train_res, cv=10, scoring='accuracy')

In [65]:
print("Har fold ka accuracy:", scores)
print("Average accuracy:", scores.mean())

model.fit(x_train_res, y_train_res)


Har fold ka accuracy: [0.80072464 0.76207729 0.76207729 0.82608696 0.88043478 0.89975845
 0.89359129 0.88996372 0.89721886 0.91051995]
Average accuracy: 0.8522453253421658


Pipeline(steps=[('classifier',
                 VotingClassifier(estimators=[('rf',
                                               RandomForestClassifier(class_weight='balanced')),
                                              ('xg',
                                               XGBClassifier(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             device=None,
                                                             early_stopping_rounds=None,
                                                             enable_categorical=False,
                                                             eval_metric=None,
                                                             feature_types=None,
                                                             featu...
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=None,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None, ...)),
                                              ('svc',
                                               SVC(class_weight='balanced'))]))])

In [66]:
import pickle
with open ('model_churn_pikle.pkl','wb') as f:
    pickle.dump(model,f)

with open ('process_churn_pikle.pkl','wb') as f:
    pickle.dump(process,f)

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.pipeline import Pipeline


# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import SVC  # Support Vector Classifier
# from sklearn.ensemble import VotingClassifier
# from collections import Counter
# from xgboost import XGBClassifier



# or_catagory=[['Month-to-month','One year' ,'Two year']]
# num_pipe= Pipeline(steps=[
#     ('scale',StandardScaler()
#     )])
# list_numcol=['tenure',  'MonthlyCharges','MonthlyCharges',]
# list_catcol=['gender','Partner', 'Dependents', 'PhoneService','MultipleLines', 'InternetService', 'OnlineSecurity',
#     'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
#     'StreamingMovies', 'PaperlessBilling', 'PaymentMethod']
# lable_pipe=Pipeline(
#     steps=[('label',OrdinalEncoder(categories=or_catagory))
#     ])
# cat_pipe=Pipeline(
#     steps=[
#         ('encode',OneHotEncoder(drop='first',sparse_output=False)),])


# process=ColumnTransformer(
#     transformers=[
#         ('num',num_pipe,list_numcol),
#         ('lab', lable_pipe,['Contract']),
#         ('cat',cat_pipe,list_catcol)
        
#     ])



# x_train_processed = process.fit_transform(x_train)

# smote = SMOTE(random_state=42)
# x_train_res, y_train_res = smote.fit_resample(x_train_processed, y_train)




# counter = Counter(y_train_res)
# ratio = counter[0] / counter[1]  
# model_xgboost = XGBClassifier(scale_pos_weight=ratio)#
# model_rf=  RandomForestClassifier(class_weight="balanced")
# model_svm=SVC(class_weight="balanced")


# from sklearn.ensemble import VotingClassifier
# voting_clf = VotingClassifier(estimators=[
#     ('rf', model_rf), ('xg', model_xgboost), ('svc', model_svm)],
#     voting='hard')  


# voting_clf.fit(x_train_res, y_train_res)

# model = Pipeline(steps=[
#     ("process",process),
#     ("classifier",voting_clf)
# ])
# scores = cross_val_score(voting_clf ,x_train_res, y_train_res, cv=10, scoring='accuracy')

# print("Har fold ka accuracy:", scores)
# print("Average accuracy:", scores.mean())
# x_train_res = pd.DataFrame(x_train_res, columns=x_train.columns)  # 🔥 this line is key

# model.fit(x_train, y_train)
# y_pred = model.predict(process.transform(x_test))  


# print(classification_report(y_test, y_pred))
# # random forest 78
# # xg boost      79
# #svm            76
# #knn            71

Har fold ka accuracy: [0.79830918 0.75724638 0.76086957 0.83333333 0.89009662 0.90096618
 0.89721886 0.89117291 0.89842805 0.90689238]
Average accuracy: 0.8534533468856059


ValueError: Shape of passed values is (8276, 28), indices imply (8276, 19)